In [9]:
from scipy.fft import fft2, fftshift, ifft2
from skimage.transform import resize
import numpy as np
import imageio as io
import os
import random
from scipy.ndimage import rotate


def smoothstep(edge1, edge2, x):
    t = (x - edge1) / (edge2 - edge1)
    t *= (t > 0)
    t = (t < 1) * t + (t > 1)
    return t * t * (3.0 - 2.0 * t)

def fft_masked(im, r1=0, r2=0.14):
    xx = np.arange(im.shape[-1]) / im.shape[-1]
    xx = xx * 2 - 1

    x_mesh, y_mesh = np.meshgrid(xx, xx)
    # mask = np.zeros(im.shape)
    # mask = np.sqrt(x_mesh**2 + y_mesh**2) < r2
    mask = np.sqrt(x_mesh**2 + y_mesh**2)
    mask = 1 - smoothstep(r1, r2, mask)
    # print(mask.shape)

    im_masked = np.zeros(im.shape)
    # for b in range(im.shape[0]):
    for i in range(11):
    #for i in range(3):
        im_fft = fftshift(fft2(im[i,:,:]))
        im_fft_masked = im_fft * mask
        im_masked[i,:,:] = np.abs(ifft2(im_fft_masked))
    
    return im_masked


In [ ]:
# Compiling images and mask into a single .npy file per location
mask_name = 'project_data/masks/mask_png/well'
test_data  = []
p = 0
im = np.zeros((12,1024,1024))

well_dir_path = 'project_data/brightfield/Alexa488_Fibroblasts_well7_135locations'
lense = 's001'
q = 1
s = 0
for im_path in os.listdir(well_dir_path):
    # print(z)
    # print(im_path)
    if lense in im_path:
        print(f'current lenses: {lense}')
        im_temp = io.v3.imread(well_dir_path + '/' + im_path)
        print(f'We read image {im_path}')
        im[s,:,:] = im_temp
        print(f'we saved image with s = {s}')
        s += 1
        # print(s)
        # print(lense)
    else:
        mask_name = 'project_data/masks/mask_png/well7_' + lense + '_mask.png'
        mask = io.v3.imread(mask_name)
        im[11,:,:] = mask
        np.save('project_data/brightfield/modified_135b/' +'well7_' + lense + '.npy',im)
        q += 1
        if q < 10:
            lense = 's00' + str(q)
        elif q < 100:
             lense = 's0' + str(q)
        else:
            lense = 's' + str(q) 
        im_temp = io.v3.imread(well_dir_path + '/' + im_path)
        print(f'We read image {im_path}')
        im[0,:,:] = im_temp
        print(f'we saved image with s = {0}')
        s = 1


In [3]:
# Setting the data path
abs_path = os.path.join(os.getcwd(), "brightfield")

# Paths to various wells
path_suffixes = ["50", "200a", "200b", "225a", "225b", "135a", "135b"]
paths = [abs_path + r"\modified_" + suffix for suffix in path_suffixes]


In [ ]:
# FFT-filtering and resizing images
new_size = 64

for path in paths:
    # Create new directories for resized images
    new_path = path + "_" + str(new_size)
    
    dir_exists = os.path.exists(new_path)
    if not dir_exists:
        os.makedirs(new_path)
        print(f"Created missing directory:\n {new_path}\n")
    else:
        print(f"Using existing directory:\n {new_path}\n")
    
    # Going through each image in the directory
    for name in os.listdir(path):
        im_all = np.load(os.path.join(path, name))
        
        im = im_all[:-1, :, :]
        im = fft_masked(im)
        im = resize(im, (11, new_size, new_size), anti_aliasing=True)
        
        mask = im_all[-1, :, :]
        mask = resize(mask, (new_size, new_size), anti_aliasing=False, order=0)
        
        im_final = np.zeros((12, new_size, new_size))
        im_final[:-1, :, :] = im
        im_final[-1, :, :] = mask
        
        print(name)  # Keeping track of progress
        
        np.save(os.path.join(new_path, name), im_final)


In [ ]:
# Data augmentation
DATA_PATH = abs_path + r"\modified_135a_64\\"

# Flipping the images
num_images = len(os.listdir(DATA_PATH))
for name in os.listdir(DATA_PATH):
    if name.endswith('.npy'):
        data = np.load(DATA_PATH + name)
        num_images += 1
        
        dataVF = np.flip(data, axis = 1)
        dataHF = np.flip(data, axis = 2)
        dataHVF = np.flip(dataVF, axis = 2)

        VFName = name.replace('.npy', '_VF.npy')      # Vertical flip
        HFName = name.replace('.npy', '_HF.npy')      # Horizontal flip
        HVFName = name.replace('.npy', '_HVF.npy')    # Horizontal and vertical flip
        np.save(DATA_PATH + VFName, dataVF)
        np.save(DATA_PATH + HFName, dataHF)
        np.save(DATA_PATH + HVFName, dataHVF)

# Rotating the previously flipped (and original) images
num_images = len(os.listdir(DATA_PATH)) 
for name in os.listdir(DATA_PATH):
    if name.endswith('.npy'):
        data = np.load(DATA_PATH + name)
        num_images += 1
        nameNew = name.replace('.npy', '_rot.npy')
        theta = random.randint(-20,20) # Only rotates between -20 and 20 degrees
        dataRotated = np.zeros(data.shape)
        for i in range(data.shape[0]):
            if i == 11:
                padded_image = np.pad(data[i,:,:], pad_width=10, mode='reflect')  # Pad before rotation
                rotated_image = rotate(padded_image, angle=theta, reshape=False, order=1, prefilter=False)
                dataRotated[i,:,:] = rotated_image[10:-10, 10:-10]  # Crop to original size

            else:
                padded_image = np.pad(data[i,:,:], pad_width=10, mode='reflect')  # Pad before rotation
                rotated_image = rotate(padded_image, angle=theta, reshape=False, order=1, prefilter=False)
                dataRotated[i,:,:] = rotated_image[10:-10, 10:-10]  # Crop to original size
        np.save(DATA_PATH + nameNew, dataRotated)
